In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./data/stations/modified_stations.csv")

In [3]:
df.head()

,id,latitude,longitude,elevation,state,name,gsn_flag,hcn_crn_flag,wmo_id
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0


In [4]:
rows_with_empty_id = df[df['id'] == ""]
rows_with_empty_id

,id,latitude,longitude,elevation,state,name,gsn_flag,hcn_crn_flag,wmo_id


In [28]:
import psycopg2

def populate_db():
    try:
        # Connect to your postgres DB
        connection = psycopg2.connect(
            dbname="mydatabase",
            user="myuser",
            password="mypassword",
            host="localhost",  # Connect to the host where Docker is running
            port="5432"
        )
        cursor = connection.cursor()

        # print(df.head())

        cursor.execute('''
                    CREATE TABLE IF NOT EXISTS Station (
                        id VARCHAR(100) PRIMARY KEY,
                        latitude FLOAT,
                        longitude FLOAT,
                        elevation FLOAT,
                        state VARCHAR(100),
                        name VARCHAR(100),
                        gsn_flag VARCHAR(100),
                        hcn_crn_flag VARCHAR(100),
                        wmo_id VARCHAR(100)
                    )
                ''')
        
        for index, row in df.iterrows():
            cursor.execute('''
                INSERT INTO Station 
                           (
                            id,
                            latitude,
                            longitude,
                            elevation,
                            state,
                            name,
                            gsn_flag,
                            hcn_crn_flag,
                            wmo_id
                           ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            ''', (
                row['id'], row['latitude'],row['longitude'],row['elevation'], 
                row['state'],row['name'],row['gsn_flag'],row['hcn_crn_flag'],row['wmo_id']))

        # Commit changes
        connection.commit()

    except Exception as error:
        print(f"Error: {error}")
    
    finally:
        if connection:
            cursor.close()
            connection.close()

if __name__ == "__main__":
    populate_db()


In [33]:

try:
    # Connect to your postgres DB
    connection = psycopg2.connect(
        dbname="mydatabase",
        user="myuser",
        password="mypassword",
        host="localhost",  # Connect to the host where Docker is running
        port="5432"
    )
    cursor = connection.cursor()

    # print(df.head())

    res = cursor.execute('''
                SELECT COUNT(*) FROM Station
            ''')
    print("Test")
    print(res)

except Exception as error:
    print(f"Error: {error}")

finally:
    if connection:
        cursor.close()
        connection.close()

Test
None


In [8]:
# from database.models import Station
# for index, row in df.iterrows():
#     obj = Station(
#         id=row['id'],
#         latitude=row['latitude'],
#         longitude=row['longitude'],
#         elevation=row['elevation'],
#         state=row['state'],
#         name=row['name'],
#         gsn_flag=row['gsn_flag'],
#         hcn_crn_flag=row['hcn_crn_flag'],
#         wmo_id=row['wmo_id'],
#         # assign other fields accordingly
#     )
#     obj.save()

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.